In [9]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd
import sys
import csv

def get_currency_rates_for_month(session, month, currency_code):
    first_day = month.replace(day=1)
    next_month = (first_day + timedelta(days=32)).replace(day=1)
    last_day = next_month - timedelta(days=1)

    url = f"http://cbr.ru/scripts/XML_dynamic.asp?date_req1={first_day.strftime('%d/%m/%Y')}&date_req2={last_day.strftime('%d/%m/%Y')}&VAL_NM_RQ={currency_code}"

    response = session.get(url)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        rates = []
        for valute in root.findall('.//Record'):
            date = valute.attrib['Date']
            value = float(valute.find('Value').text.replace(',', '.'))
            nominal = float(valute.find('Nominal').text)
            rate = value / nominal
            rates.append({'date': date, 'rate': rate})
        return rates
    else:
        print(f"Failed to retrieve currency rates for {currency_code}.")
        return []

def main():
    current_month = datetime.today().replace(day=1)
    currency_codes = ["R01235", "R01239", "R01375"]
    currency_names = ["Доллар США", "Евро", "Китайский юань"]

    data = []
    with requests.Session() as session:
        for currency_code, currency_name in zip(currency_codes, currency_names):
            rates = get_currency_rates_for_month(session, current_month, currency_code)
            for rate in rates:
                data.append([rate['date'], currency_name, rate['rate']])

    df = pd.DataFrame(data, columns=["date", "currency", "rate"])

    df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')

    df = df.sort_values(by='date')

    df.reset_index(drop=True, inplace=True)

    df.to_csv(sys.stdout, index=False, encoding='UTF-8', quoting=csv.QUOTE_NONNUMERIC, sep=";")

if __name__ == "__main__":
    main()

"date";"currency";"rate"
"2024-06-01";"Доллар США";90.1915
"2024-06-01";"Евро";97.7908
"2024-06-01";"Китайский юань";12.4028
"2024-06-04";"Доллар США";89.3755
"2024-06-04";"Евро";96.8721
"2024-06-04";"Китайский юань";12.3011
"2024-06-05";"Доллар США";88.7574
"2024-06-05";"Евро";96.563
"2024-06-05";"Китайский юань";12.2248
"2024-06-06";"Евро";96.4443
"2024-06-06";"Китайский юань";12.2244
"2024-06-06";"Доллар США";88.7436
"2024-06-07";"Китайский юань";12.2251
"2024-06-07";"Доллар США";88.7604
"2024-06-07";"Евро";96.5698
"2024-06-08";"Китайский юань";12.229
"2024-06-08";"Доллар США";88.7606
"2024-06-08";"Евро";96.6916
"2024-06-11";"Китайский юань";12.2213
"2024-06-11";"Евро";95.6367
"2024-06-11";"Доллар США";88.9944
"2024-06-12";"Евро";95.7391
"2024-06-12";"Китайский юань";12.231
"2024-06-12";"Доллар США";89.0214
"2024-06-14";"Евро";94.8324
"2024-06-14";"Китайский юань";12.0408
"2024-06-14";"Доллар США";88.208
"2024-06-15";"Китайский юань";12.1629
"2024-06-15";"Доллар США";89.0658
"2024-0

In [10]:
df

,date,currency,rate
0,2024-06-01,Доллар США,90.1915
1,2024-06-01,Евро,97.7908
2,2024-06-01,Китайский юань,12.4028
3,2024-06-04,Доллар США,89.3755
4,2024-06-04,Евро,96.8721
5,2024-06-04,Китайский юань,12.3011
6,2024-06-05,Доллар США,88.7574
7,2024-06-05,Евро,96.5630
8,2024-06-05,Китайский юань,12.2248
9,2024-06-06,Евро,96.4443
